In [ ]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import os
from PIL import Image
from IPython.display import display

# Load datasets
email_details = pd.read_csv("./CSV/email_thread_details.csv")
email_summaries = pd.read_csv("./CSV/email_thread_summaries.csv")
fashion_data = pd.read_csv("./CSV/Fashion Dataset v2.csv")

# Initialize OpenAI API Key
openai.api_key = os.getenv("")

# Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


/Users/snehaljadhav/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/snehaljadhav/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Function to create FAISS index
def create_faiss_index(data, text_column):
    vectors = embedding_model.encode(data[text_column].tolist(), convert_to_numpy=True)
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    return index, vectors

# Create FAISS indexes
email_index, email_vectors = create_faiss_index(email_summaries, "summary")
emaildetails_index, emaildetails_vectors = create_faiss_index(email_details, "body")
fashion_index, fashion_vectors = create_faiss_index(fashion_data, "name")


In [ ]:
# Function to search in FAISS index
def search_faiss(index, query, data, k=5):
    query_vector = embedding_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, k)
    return data.iloc[indices[0]]

# Fine-tuned function to generate OpenAI response
def generate_fine_tuned_response(context, query, domain):
    openai.api_key = ""  # Replace with actual API key
    if domain == "email":
        role = "You are an expert AI assistant providing precise summaries of email discussions."
        task = "Extract key insights and provide concise summaries of the given email threads."
    elif domain == "fashion":
        role = "You are a fashion recommendation AI specializing in personalized clothing suggestions."
        task = "Provide detailed fashion insights and recommend similar products based on user queries."
    else:
        role = "You are an AI assistant handling diverse information retrieval tasks."
        task = "Answer the query accurately using the provided context."

    messages = [
        {"role": "system", "content": f"{role} {task}"},
        {"role": "user", "content": f"Query: {query}\nContext: {context}"}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7
    )
    return response["choices"][0]["message"]["content"]


In [5]:
# Example queries for email search
email_queries = [
    "Natural Gas deals.",
    "Any Escalations to highlight.",
    "Any Orders on power plant.",
    "Any Discussion on Liquidity metrics",
    "Are there any challenges w.r.t counterparties",
    "Summarize discussions on AI research in the last year.",
    "What were the key points in the last cybersecurity meeting?",
    "Summarize project updates from Q4.",
    "List top software development challenges discussed recently.",
    "What were the main decisions in the last board meeting?"
]

for query in email_queries:
    results = search_faiss(email_index, query, email_summaries)
    context = "\n".join(results["summary"].tolist())
    response = generate_fine_tuned_response(context, query, "email")
    print(f"Query: {query}\nResponse: {response}\n{'-'*50}")




Query: Natural Gas deals.
Response: The email thread discusses various aspects of natural gas deals. It starts with a request for original tickets for deals with Northern Natural Gas and further discusses specifics about deals with Duke/Centana, Ponderosa Energy Partners, Pace Energy, and General Land Office. There's also a discussion on the impact of a natural gas deal in California on the market, with many customers waiting for market stabilization before hedging. 

A study by the Center for the Advancement of Energy Markets (CAEM) highlighting the benefits of gas deregulation is mentioned, citing savings of over $600 billion for consumers in nearly 20 years. 

The thread also delves into the current state of the natural gas market with differing opinions on the increase in gas production and potential decrease in demand. 

Lastly, it includes a debate on natural gas prices and trading strategies for natural gas and heating oil contracts, with varying viewpoints presented by Jennifer

In [6]:
import requests

In [7]:
# Example queries for fashion search
fashion_queries = [
    "Show me red dresses.",
    "Find me sports shoes for men.",
    "Show stylish winter jackets.",
    "I need formal shirts.",
    "Find traditional Indian wear.",
    "Find traditional Indian Sarees in blue and golden color.",
    "Find traditional Indian Embroidery Cotton Kurta."
]

for query in fashion_queries:
    results = search_faiss(fashion_index, query, fashion_data)
    display(results[["name", "img"]])
    context = "\n".join(results["name"].tolist())
    response = generate_fine_tuned_response(context, query, "fashion")
    print(f"Query: {query}\nResponse: {response}\n{'-'*50}")
    for _, row in results.iterrows():
        print(f"Product: {row['name']}")
        # Uncomment this line and you will get the images displayed, commented this because size of the python file is about 6O MB and cant upload beyond 50MB limitation
        #img = Image.open(requests.get(row["img"], stream=True).raw)
        #display(img)

,name,img
5014,Dress My Angel Girls Red Printed Mini-Length S...,http://assets.myntassets.com/assets/images/187...
4402,Go Colors Women Red Tapered Fit Trousers,http://assets.myntassets.com/assets/images/182...
7193,Sera Red Two Piece Party Dress,http://assets.myntassets.com/assets/images/164...
7304,DressBerry Women Red Top with Trousers,http://assets.myntassets.com/assets/images/142...
5092,max Women Red Printed Skirts,http://assets.myntassets.com/assets/images/186...


Query: Show me red dresses.
Response: Based on your preference for red dresses, here are some similar product recommendations:

1. "Forever 21 Women's Red Bodycon Dress": This red bodycon dress from Forever 21 is perfect for any special occasion. The deep red color and fitted silhouette highlight your curves, while the mini-length adds a playful touch.

2. "FabAlley Women's Red Skater Dress": This red skater dress by FabAlley is a great choice for a casual day out. The mini-length and flared skirt create a fun, flirty look that pairs well with flats or heels.

3. "Zara Women's Red Midi Dress": This red midi dress from Zara offers a more sophisticated look. The mid-length cut and structured silhouette make it a great choice for work or a formal event.

4. "H&M Women's Red A-Line Dress": This A-line dress from H&M is versatile and stylish. The bright red color is eye-catching, and the A-line silhouette is flattering on all body types.

5. "Levi's Women's Red Shirt Dress": This red shirt 

,name,img
3759,AGIL ATHLETICA Women Navy Blue Solid Regular F...,http://assets.myntassets.com/assets/images/pro...
3697,Sweet Dreams Women Black Solid Regular Fit Spo...,http://assets.myntassets.com/assets/images/pro...
3232,Revolution Women Plus Size Black Solid Pure Co...,http://assets.myntassets.com/assets/images/129...
4349,KASSUALLY Women Navy Blue Regular Fit Solid Bo...,http://assets.myntassets.com/assets/images/pro...
2998,Nike Women Purple Loose Fit High Rise FLY CROS...,http://assets.myntassets.com/assets/images/189...


Query: Find me sports shoes for men.
Response: Based on your query, I understand that you're looking for men's sports shoes. Unfortunately, the context provided includes only women's clothing items, and doesn't provide any information about men's sports shoes.

However, I can make some general suggestions based on popular and highly rated men's sports shoes:

1. **Nike Men's Revolution 5 Running Shoe**: This is a versatile running shoe with a minimalist design, soft foam midsole, and a durable rubber outsole. It provides a lightweight and comfortable fit tailored for running but can also be suitable for various sports activities.

2. **Adidas Men's Ultraboost 21 Running Shoe**: This shoe is known for its high-performance features like responsive cushioning, supportive heel counter, and a flexible outsole. It's perfect for running and other sports activities.

3. **Under Armour Men's Charged Assert 8 Running Shoe**: This shoe is a great choice for sports due to its lightweight mesh uppe

,name,img
12610,iki chic Women Stylish Black Washed Jacket,http://assets.myntassets.com/assets/images/155...
12368,HRX By Hrithik Roshan Outdoor Women Winter Mos...,http://assets.myntassets.com/assets/images/146...
11876,BUY NEW TREND Women Beige Camouflage Lightweig...,http://assets.myntassets.com/assets/images/176...
12171,URBANIC Women Black Tailored Jacket,http://assets.myntassets.com/assets/images/156...
12225,BUY NEW TREND Women Black Washed Lightweight C...,http://assets.myntassets.com/assets/images/176...


...

Query: Find traditional Indian wear.
Response: Based on your interest in traditional Indian wear, I've found a few similar options that might catch your eye, all of which are from brands known for their ethnic Indian designs.

1. [PISARA Women Turquoise Blue Ethnic Motifs Anarkali Kurta with Trousers & Dupatta](https://www.example.com) - This set echoes the turquoise blue color and floral design of the AMIRAS ensemble, but with an Anarkali silhouette for added flair.

2. [VASTRANAND Women Black & Gold-Toned Embroidered Straight Kurta with Trousers & Dupatta](https://www.example.com) - These match the black color and embroidered details of the SWAGG INDIA trousers, but in a full set for a coordinated look.

3. [BIBA Women Peach-Coloured Embroidered Anarkali Kurta with Trousers & Dupatta](https://www.example.com) - A set similar to the SWAGG INDIA peach-colored trousers, but this time with an embroidered Anarkali kurta to complete the ensemble.

4. [INDYA Women Green & Gold-Toned Yoke De

,name,img
8535,Soch Blue & Gold-Toned Floral Saree,http://assets.myntassets.com/assets/images/191...
8349,Sangria Women Blue Sarees,http://assets.myntassets.com/assets/images/184...
8812,Sangria Women Turquoise Blue Sarees,http://assets.myntassets.com/assets/images/184...
8770,Soch Red & Gold-Toned Ethnic Motifs Sequinned ...,http://assets.myntassets.com/assets/images/163...
8786,Varkala Silk Sarees Turquoise Blue Kanjeevaram...,http://assets.myntassets.com/assets/images/177...


Query: Find traditional Indian Sarees in blue and golden color.
Response: Based on your query, here are some recommendations:

1. **Soch Blue & Gold-Toned Floral Saree:** This saree features beautiful floral motifs in gold against a rich blue background. It's perfect for traditional occasions and can be paired with gold-toned jewelry to enhance the look.

2. **Sangria Women Blue Sarees:** The Sangria Women Blue Saree is a versatile choice. It can be dressed up with traditional gold jewelry for a festive occasion or worn with simple accessories for a more casual look.

3. **Sangria Women Turquoise Blue Sarees:** This saree is in a stunning shade of turquoise blue, offering a unique twist to traditional Indian sarees. Pair it with gold-toned accessories for a stylish and elegant look.

4. **Soch Red & Gold-Toned Ethnic Motifs Sequinned Saree:** Even though this saree is red, its gold-toned ethnic motifs and sequinned detailing make it a standout piece. It's perfect for those looking for 

,name,img
127,Ishin Women Teal Embroidered Regular Sequinned...,http://assets.myntassets.com/assets/images/155...
865,Indo Era Embroidered Velvet Kurta Set,http://assets.myntassets.com/assets/images/157...
413,Ishin Women White Ethnic Motifs Embroidered La...,http://assets.myntassets.com/assets/images/pro...
7823,ADA Women White Chikankari Hand Embroidered Pu...,http://assets.myntassets.com/assets/images/115...
901,SWAGG INDIA Women Green Floral Embroidered Reg...,http://assets.myntassets.com/assets/images/156...


Query: Find traditional Indian Embroidery Cotton Kurta.
Response: Based on your query, here are some similar product recommendations:

1. [Ishin Women's Yellow Embroidered Regular Sequinned Pure Cotton Kurta with Palazzo & With Dupatta](https://www.amazon.in/dp/B08X1J9ZKX/): This ensemble is similar to your Ishin Teal Embroidered set, providing the same comfort and style, but in a vibrant yellow color.

2. [Indo Era Women's Pink Embroidered Velvet Kurta Set](https://www.amazon.in/dp/B08XJG1FGK/): This set mirrors the elegance of the Indo Era Velvet Kurta set you liked, but features a different color palette.

3. [Ishin Women's Red Ethnic Motifs Embroidered Layered Pure Cotton Kurta with Patiala & With Dupatta](https://www.amazon.in/dp/B08X6SZZV1/): Similar to the Ishin White Ethnic Motifs set, this ensemble features a red hue and a different style of trousers.

4. [ADA Women's Blue Chikankari Hand Embroidered Pure Cotton A-Line Sustainable Kurti](https://www.amazon.in/dp/B08X6SZZV1/): 

# Metadata-Based Filtering for Fashion and Email Datasets

##  Metadata Filtering for `p_attributes` in Fashion Dataset

In [8]:

import ast

# Function to filter fashion search results based on metadata, including p_attributes
def filter_fashion_results(indices, min_price=None, max_price=None, product_type=None, fabric=None, pattern=None, occasion=None, sleeve_length=None, neck_type=None):
    filtered_results = []
    for idx in indices[0]:
        row = fashion_data.iloc[idx]
        
        # Convert p_attributes string to dictionary if it's stored as a string
        p_attributes = row["p_attributes"]
        if isinstance(p_attributes, str):
            try:
                p_attributes = ast.literal_eval(p_attributes)  # Safely convert string to dictionary
            except (ValueError, SyntaxError):
                p_attributes = {}

        # Debugging: Print extracted metadata for verification
        print(f"Processing Product: {row['name']}")
        print(f"Extracted p_attributes: {p_attributes}")

        # Apply filtering conditions
        if (min_price is None or row["price"] >= min_price) and            (max_price is None or row["price"] <= max_price) and            (product_type is None or p_attributes.get("Top Type", "").lower() == product_type.lower()) and            (fabric is None or p_attributes.get("Top Fabric", "").lower() == fabric.lower()) and            (pattern is None or p_attributes.get("Top Pattern", "").lower() == pattern.lower()) and            (occasion is None or p_attributes.get("Occasion", "").lower() == occasion.lower()) and            (sleeve_length is None or p_attributes.get("Sleeve Length", "").lower() == sleeve_length.lower()) and            (neck_type is None or p_attributes.get("Neck", "").lower() == neck_type.lower()):
            filtered_results.append(row.to_dict())
            print("✔ Match Found: Added to Results")

    if not filtered_results:
        print("❌ No Matches Found Based on Metadata Filtering Conditions")

    return filtered_results




# import ast

# # Function to filter fashion search results based on metadata, ensuring p_attributes is correctly parsed
# def filter_fashion_results(indices, min_price=None, max_price=None, product_type=None, fabric=None, pattern=None, occasion=None, sleeve_length=None, neck_type=None):
#     filtered_results = []
#     for idx in indices[0]:
#         row = fashion_data.iloc[idx]

#         # Convert p_attributes string to dictionary if stored incorrectly
#         p_attributes = row["p_attributes"]
#         if isinstance(p_attributes, str):
#             try:
#                 p_attributes = ast.literal_eval(p_attributes)  # Safely convert string to dictionary
#             except (ValueError, SyntaxError):
#                 p_attributes = {}

#         # Debugging: Print extracted metadata for verification
#         print(f"Processing Product: {row['name']}")
#         print(f"Extracted p_attributes: {p_attributes}")

#         # Apply filtering conditions
#         if (min_price is None or row["price"] >= min_price) and            (max_price is None or row["price"] <= max_price) and            (product_type is None or p_attributes.get("Top Type", "").lower() == product_type.lower()) and            (fabric is None or p_attributes.get("Top Fabric", "").lower() == fabric.lower()) and            (pattern is None or p_attributes.get("Top Pattern", "").lower() == pattern.lower()) and            (occasion is None or p_attributes.get("Occasion", "").lower() == occasion.lower()) and            (sleeve_length is None or p_attributes.get("Sleeve Length", "").lower() == sleeve_length.lower()) and            (neck_type is None or p_attributes.get("Neck", "").lower() == neck_type.lower()):
#             filtered_results.append(row.to_dict())
#             print("✔ Match Found: Added to Results")

#     if not filtered_results:
#         print("❌ No Matches Found Based on Metadata Filtering Conditions")

#     return filtered_results



##  Metadata Filtering for Email Dataset

In [9]:

import ast
import re
from datetime import datetime

# Function to clean and extract email metadata properly
def clean_email_metadata(email_row):
    sender = email_row["from"]
    recipients = ast.literal_eval(email_row["to"]) if isinstance(email_row["to"], str) else email_row["to"]
    timestamp = email_row["timestamp"]

    # Convert timestamp to datetime format
    try:
        sent_date = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        sent_date = None  # Handle parsing errors

    # Extract Cc from body if available
    cc_match = re.search(r'Cc:\s*(.*)', email_row["body"], re.DOTALL)
    cc_recipients = cc_match.group(1).strip().split(";") if cc_match else []
    
    # Combine To and Cc recipients
    all_recipients = recipients + cc_recipients

    # Debugging Output
    print(f"Extracted Sender: {sender}")
    print(f"Extracted Sent Date: {sent_date}")
    print(f"Extracted Recipients (To + Cc): {all_recipients}")

    return sender, sent_date, all_recipients

# Function to filter email search results based on metadata with recipient & timestamp handling
def filter_email_results(indices, sender=None, receiver=None, start_date=None, end_date=None):
    filtered_results = []
    
    print("Debugging Email Metadata Filtering:")
    print(f"Sender Filter: {sender}, Receiver Filter: {receiver}, Start Date: {start_date}, End Date: {end_date}")

    for idx in indices[0]:
        row = email_details.iloc[idx]

        # Clean and extract metadata from the row
        extracted_sender, extracted_sent_date, extracted_recipients = clean_email_metadata(row)

        # Debugging: Show row before filtering
        print(f"Checking Email: {row['subject']}")

        # Apply filtering conditions
        if (sender is None or sender.lower() in extracted_sender.lower()) and            (receiver is None or any(receiver.lower() in r.lower() for r in extracted_recipients)) and            (start_date is None or (extracted_sent_date and extracted_sent_date >= datetime.strptime(start_date, "%Y-%m-%d"))) and            (end_date is None or (extracted_sent_date and extracted_sent_date <= datetime.strptime(end_date, "%Y-%m-%d"))):
            filtered_results.append(row.to_dict())
            print("✔ Match Found: Added to Results")

    if not filtered_results:
        print("❌ No Matches Found Based on Metadata Filtering Conditions")

    return filtered_results


##  Search Functions with Metadata Filtering

In [10]:

# Function to perform filtered search for Fashion dataset
def search_fashion(query, k=5, min_price=None, max_price=None, product_type=None, fabric=None, pattern= None, occasion= None, sleeve_length=None, neck_type=None, product_name=None):
    query_vector = embedding_model.encode([query], convert_to_numpy=True)
    distances, indices = fashion_index.search(query_vector, k)
    return filter_fashion_results(indices, min_price, max_price, product_type, fabric, pattern, occasion, sleeve_length, neck_type)


# Function to perform filtered search for Email dataset
def search_email(query, k=5, sender=None, receiver=None, start_date=None, end_date=None):
    query_vector = embedding_model.encode([query], convert_to_numpy=True)
    distances, indices = email_index.search(query_vector, k)
    return filter_email_results(indices, sender, receiver, start_date, end_date)


## Example Search Queries for Metadata Filtering

In [11]:
# Email Dataset Queries
email_results_1 = search_email(
    query="project update", 
    k=5, 
    sender="manager@example.com", 
    receiver="team@example.com", 
    start_date="2024-01-01", 
    end_date="2024-02-01"
)
print("Filtered Email Search Results (Example 1):", email_results_1)

email_results_2 = search_email(
    query="meeting schedule", 
    k=5, 
    sender="ceo@example.com", 
    start_date="2023-12-01", 
    end_date="2024-01-15"
)
print("Filtered Email Search Results (Example 2):", email_results_2)

Debugging Email Metadata Filtering:
Sender Filter: manager@example.com, Receiver Filter: team@example.com, Start Date: 2024-01-01, End Date: 2024-02-01
Extracted Sender: Kay Mann
Extracted Sent Date: 2001-02-14 23:18:00
Extracted Recipients (To + Cc): ['Rusty Stevens']
Checking Email: Confidentiality Agreements
Extracted Sender: Eric Bass
Extracted Sent Date: 2001-01-05 03:51:00
Extracted Recipients (To + Cc): ['"Bass', 'Jason" <Jason.Bass2@COMPAQ.com> @ ENRON', 'shusser@enron.com\nSubject: New Years\n\n\nJason,\n\nShanna and I were wondering what you and Donnita were doing for New Years?\n\n-Eric']
Checking Email: RE: New Years
Extracted Sender: John Arnold
Extracted Sent Date: 2001-04-23 15:53:00
Extracted Recipients (To + Cc): ['Mike Maggi <Mike Maggi/Corp/Enron@Enron', 'John Griffith <John Griffith/Corp/Enron@Enron', 'John Disturnal <John Disturnal/CAL/ECT@ECT']
Checking Email: option candlesticks
Extracted Sender: Quigley, Dutch DQUIGLE
Extracted Sent Date: 2001-09-26 13:53:55
Ext

In [12]:
 #Example Queries for Metadata-Based Filtering

# Fashion Dataset Queries
fashion_results_1 = search_fashion(
    query="ethnic kurta", 
    k=5, 
    min_price=1000, 
    max_price=5000, 
    product_type="Kurta", 
    fabric="Viscose Rayon", 
    pattern="Printed", 
    occasion="Festive", 
    sleeve_length="Three-Quarter Sleeves", 
    neck_type="Mandarin Collar"
)
print("Filtered Fashion Search Results (Example 1):", fashion_results_1)

fashion_results_2 = search_fashion(
    query="casual cotton dress", 
    k=5, 
    min_price=500, 
    max_price=5000, 
    product_type="Shirt", 
    fabric="Cotton", 
    pattern="Solid", 
    occasion="Casual"
)
print("Filtered Fashion Search Results (Example 2):", fashion_results_2)

Processing Product: KALINI Women Black Ethnic Motifs Kurta
Extracted p_attributes: {'Body Shape ID': '424', 'Body or Garment Size': 'To-Fit Denotes Body Measurements in', 'Care for me': 'Follow the washcare label on garment. Use cold water with mild detergents when handwashing or opt for delicate setting when machine washing<br>\r\nAlways wash inside-out to preserve colour<br>\r\nNever wring or tumble dry. Dry flat on a towel or hang on coat hangers<br>\r\nIron on the reverse side with medium heat', 'Colour Family': 'Earthy', 'Design Styling': 'Regular', 'Fabric': 'Viscose Rayon', 'Fabric 2': 'NA', 'Fabric Purity': 'Pure', 'Hemline': 'High-Low', 'Length': 'Knee Length', 'Main Trend': 'NA', 'Neck': 'Mandarin Collar', 'Number of Pockets': 'NA', 'Occasion': 'Daily', 'Ornamentation': 'NA', 'Pattern': 'Woven Design', 'Print or Pattern Type': 'Ethnic Motifs', 'Shape': 'Straight', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve Styling': 'Regular Sleeves', 'Slit Detail': 'Front Slit', 'Stit

In [13]:
# Example Queries for Metadata-Based Filtering

print("\nExecuting Fashion Dataset Queries with Metadata Filtering:\n")

# Search for an ethnic printed Kurta in a festive occasion
fashion_results_1 = search_fashion(
    query="ethnic kurta", 
    k=5, 
    min_price=1000, 
    max_price=5000, 
    product_type="Kurta", 
    fabric="Viscose Rayon", 
    pattern="Printed", 
    occasion="Festive", 
    sleeve_length="Three-Quarter Sleeves", 
    neck_type="Mandarin Collar"
)
print("Filtered Fashion Search Results (Example 1):", fashion_results_1)

# Search for a casual cotton solid dress
fashion_results_2 = search_fashion(
    query="casual cotton dress", 
    k=5, 
    product_type="Dress", 
    fabric="Cotton", 
    pattern="Solid", 
    occasion="Casual"
)
print("Filtered Fashion Search Results (Example 2):", fashion_results_2)





Executing Fashion Dataset Queries with Metadata Filtering:

Processing Product: KALINI Women Black Ethnic Motifs Kurta
Extracted p_attributes: {'Body Shape ID': '424', 'Body or Garment Size': 'To-Fit Denotes Body Measurements in', 'Care for me': 'Follow the washcare label on garment. Use cold water with mild detergents when handwashing or opt for delicate setting when machine washing<br>\r\nAlways wash inside-out to preserve colour<br>\r\nNever wring or tumble dry. Dry flat on a towel or hang on coat hangers<br>\r\nIron on the reverse side with medium heat', 'Colour Family': 'Earthy', 'Design Styling': 'Regular', 'Fabric': 'Viscose Rayon', 'Fabric 2': 'NA', 'Fabric Purity': 'Pure', 'Hemline': 'High-Low', 'Length': 'Knee Length', 'Main Trend': 'NA', 'Neck': 'Mandarin Collar', 'Number of Pockets': 'NA', 'Occasion': 'Daily', 'Ornamentation': 'NA', 'Pattern': 'Woven Design', 'Print or Pattern Type': 'Ethnic Motifs', 'Shape': 'Straight', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve St

In [14]:
print("\nExecuting Email Dataset Queries with Metadata Filtering:\n")

# Search for project update emails from a specific sender to a team with a date range
email_results_1 = search_email(
    query="project update", 
    k=5, 
    sender="manager@example.com", 
    receiver="team@example.com", 
    start_date="2024-01-01", 
    end_date="2024-02-01"
)
print("Filtered Email Search Results (Example 1):", email_results_1)

# Search for meeting schedule emails from a CEO within a specific date range
email_results_2 = search_email(
    query="meeting schedule", 
    k=5, 
    sender="ceo@example.com", 
    start_date="2023-12-01", 
    end_date="2024-01-15"
)
print("Filtered Email Search Results (Example 2):", email_results_2)


Executing Email Dataset Queries with Metadata Filtering:

Debugging Email Metadata Filtering:
Sender Filter: manager@example.com, Receiver Filter: team@example.com, Start Date: 2024-01-01, End Date: 2024-02-01
Extracted Sender: Kay Mann
Extracted Sent Date: 2001-02-14 23:18:00
Extracted Recipients (To + Cc): ['Rusty Stevens']
Checking Email: Confidentiality Agreements
Extracted Sender: Eric Bass
Extracted Sent Date: 2001-01-05 03:51:00
Extracted Recipients (To + Cc): ['"Bass', 'Jason" <Jason.Bass2@COMPAQ.com> @ ENRON', 'shusser@enron.com\nSubject: New Years\n\n\nJason,\n\nShanna and I were wondering what you and Donnita were doing for New Years?\n\n-Eric']
Checking Email: RE: New Years
Extracted Sender: John Arnold
Extracted Sent Date: 2001-04-23 15:53:00
Extracted Recipients (To + Cc): ['Mike Maggi <Mike Maggi/Corp/Enron@Enron', 'John Griffith <John Griffith/Corp/Enron@Enron', 'John Disturnal <John Disturnal/CAL/ECT@ECT']
Checking Email: option candlesticks
Extracted Sender: Quigley,